In [1]:
import tensorflow as tf 

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [3]:
data_x, data_y = [], []
num_id = 3
img_length = 400*400*3

for i in range(num_id):
    part_x, part_y = [], []
    folder = 'pixiv/part_0'+str(i)
    for img_file in os.listdir(folder):
#         print(folder+'/'+img_file)
        img = np.array(Image.open(folder+'/'+img_file))
        img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
        if(len(img)>img_length):
            img = np.array([img[j] for j in range(len(img)) if j%4!=3])
            print(len(img))
#         data_x.append([j/255.0 for j in img])
        part_x.append(img/255.0)
        
        label = [0 for j in range(0, num_id)]
        label[i]=1
        part_y.append(label)   
        if(len(part_x)>=800):
            break
    print(len(part_x))
    data_x.extend(part_x)
    data_y.extend(part_y)
    print(folder+' is completed.')

print(len(data_x))
print(len(data_y))

800
pixiv/part_00 is completed.
800
pixiv/part_01 is completed.
800
pixiv/part_02 is completed.
2400
2400


In [4]:
data_x = np.array(data_x, dtype='float32')
data_y = np.array(data_y, dtype='float32')

print(data_x[:10])
print(data_y[:10])

[[ 0.2         0.28235295  0.29411766 ...,  0.1882353   0.27843139
   0.25490198]
 [ 0.03137255  0.12156863  0.56078434 ...,  0.02745098  0.02352941
   0.29803923]
 [ 0.25098041  0.2         0.13725491 ...,  0.1254902   0.09411765
   0.01960784]
 ..., 
 [ 0.65098041  0.52941179  0.69411767 ...,  0.17647059  0.20392157
   0.17254902]
 [ 0.03529412  0.07843138  0.08627451 ...,  0.          0.01960784
   0.00392157]
 [ 0.83529413  0.69411767  0.56862748 ...,  0.24705882  0.25882354
   0.33333334]]
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]


In [5]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=int(len(data_x)*9/10)
print(n)

train_x = [data_x[i] for i in order[0:n]]
train_y = [data_y[i] for i in order[0:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

2400
2160
2160
240


In [6]:
train_x_list, train_y_list = [], []
valid_x_list, valid_y_list = [], []

n=int(len(data_x)/10)
print(n)
for i in range(0, 9):
#     train_x_list.append([train_x[i] for i in range(0, i*n)+range((i+1)*n), len(train_x)])
    cur_x = [train_x[i] for i in range(0, i*n)]
    cur_x.extend([train_x[i] for i in range((i+1)*n, len(train_x))])
    cur_y = [train_y[i] for i in range(0, i*n)]
    cur_y.extend([train_y[i] for i in range((i+1)*n, len(train_x))])
    print(len(cur_x))
#     print(len(cur_y))
    
    train_x_list.append(cur_x)
    train_y_list.append(cur_y)
    
    valid_x_list.append([train_x[i] for i in range(i*n, (i+1)*n)])
    valid_y_list.append([train_y[i] for i in range(i*n, (i+1)*n)])

print(len(train_x_list))

240
1920
1920
1920
1920
1920
1920
1920
1920
1920
9


In [7]:
graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    labels = tf.placeholder('float', [None, num_id])            #
    inputs = tf.placeholder('float', [None, img_length])                        #输入的数据占位符
#     print(x.shape)
#     print(y_actual.shape)

    def weight_variable(shape): #定义一个函数，用于初始化所有的权值 W
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape): #定义一个函数，用于初始化所有的偏置项 b
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W): #定义一个函数，用于构建卷积层
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

    def max_pool(x): #定义一个函数，用于构建池化层
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

    #构建网络
    x_image = tf.reshape(inputs, [-1,400,400,3])         #转换输入数据shape,以便于用于网络中
    W_conv1 = weight_variable([5, 5, 3, 32])      
    b_conv1 = bias_variable([32])       
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)     #第一个卷积层
    h_pool1 = max_pool(h_conv1)                                  #第一个池化层
    print(h_pool1.shape)#(?, 200, 200, 32)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
    h_pool2 = max_pool(h_conv2)                                   #第二个池化层
    print(h_pool2.shape)#(?, 100, 100, 64)

    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)      #第二个卷积层
    h_pool3 = max_pool(h_conv3)                                   #第二个池化层
    print(h_pool3.shape)#(?, 50, 50, 128)
    
    # 第三层 是个全连接层,输入维数125*125*64, 输出维数为1024
    W_fc1 = weight_variable([50 * 50 * 128, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 50*50*128])              #reshape成向量
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)    #第一个全连接层

    keep_prob = tf.placeholder("float") 
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

    W_fc2 = weight_variable([1024, num_id])
    b_fc2 = bias_variable([num_id])
#     y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层
    y_predict=tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#     loss = -tf.reduce_sum(labels*tf.log(y_predict))     #交叉熵
#     optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)    #梯度下降法
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels = labels))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(labels,1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))    #精确度计算

(?, 200, 200, 32)
(?, 100, 100, 64)
(?, 50, 50, 128)


In [8]:
batch_start = 0
batch_size = 30
training_steps = 51

for i in range(0, 9):
    train_x = train_x_list[i]
    train_y = train_y_list[i]
    with tf.Session(graph=graph) as session:
    #     saver.restore(session, tf.train.latest_checkpoint('lstm_check'))
        tf.global_variables_initializer().run()
        print('Initialized', i)
        for step in range(training_steps): 
            if(batch_start+batch_size>=len(train_x)):
                order = [i for i in range(len(train_x))]
                random.shuffle(order)
                train_x = [train_x[i] for i in order]
                train_y = [train_y[i] for i in order]
                batch_start = 0        
            feed_x = train_x[batch_start:batch_start+batch_size]
            feed_y = train_y[batch_start:batch_start+batch_size]
            feed_dict = {inputs: feed_x, labels: feed_y, keep_prob:0.6}
            _, l, predictions, accur = session.run([optimizer, loss, labels, accuracy], feed_dict = feed_dict)
            if (step % 10 == 0):
                print('Loss at step %d: %f' % (step, l), ' accuarcy is ', accur)
    #             saver.save(session, 'lstm_check/my-model-1.ckpt', global_step=step+1000)
        valid_loss, valid_accur = session.run([loss,accuracy], feed_dict={inputs: valid_x_list[i], labels:  valid_y_list[i], keep_prob:1.0})
        print('==valid loss, ', valid_loss, ' valid accur is ', valid_accur)
        test_loss, test_accur = session.run([loss,accuracy], feed_dict={inputs: test_x, labels: test_y, keep_prob:1.0})
        print('--test loss, ', test_loss, ' test accur is ', test_accur)

Initialized 0


ResourceExhaustedError: OOM when allocating tensor with shape[30,32,400,400]
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape, Variable/read)]]

Caused by op 'Conv2D', defined at:
  File "/home/ziyi/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ziyi/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ziyi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ziyi/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ziyi/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/ziyi/anaconda3/lib/python3.6/site-packages/zmq/ev

In [29]:
print('test loss, ', test_loss, ' test accur is ', test_accur)

test loss,  2.90264  test accur is  0.583333
